In [ ]:
{
    "[STACKS]",
    "POT",     # Pot total
    "BB",      # Unité de mise : Big Blind
    "SB",      # Unité de mise : Small Blind
    "BTN",     # Bouton
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", # value to bet
    
    "[FLOP]", "[TURN]", "[RIVER]",  # Étapes du board
    # Cartes (valeurs + couleurs)
    "2s", "3s", "4s", "5s", "6s", "7s", "8s", "9s", "Ts", "Js", "Qs", "Ks", "As",  # Pique (spade)
    "2h", "3h", "4h", "5h", "6h", "7h", "8h", "9h", "Th", "Jh", "Qh", "Kh", "Ah",  # Cœur (heart)
    "2d", "3d", "4d", "5d", "6d", "7d", "8d", "9d", "Td", "Jd", "Qd", "Kd", "Ad",  # Carreau (diamond)
    "2c", "3c", "4c", "5c", "6c", "7c", "8c", "9c", "Tc", "Jc", "Qc", "Kc", "Ac",  # Trèfle (club)
   "[PREFLOP]", "[FLOP]", "[TURN]", "[RIVER]",
    "FOLD",
    "CALL",
    "CHECK",
    "BET",
    "RAISE",
    "ALLIN",
    ".", ":", "=", "\n", " ", "[", "]"  # Caractères spéciaux
}
